Idea code name: `S1P10`

In [1]:
from keras.layers import Input, Dense, LSTM, PReLU, Reshape, Dropout
from keras.models import model_from_json, Model
import keras

from s1p10_model.resnet8_body import resnet8_body

from s1p10_training_utils import save_lstm, DataGenerator

import json
import time

Using TensorFlow backend.


# Global Config

## Load weight of each angle class

In [2]:
with open('./s1p10_data/s1p10_classes_weight.json', 'r') as fp:
    classes_weight = json.load(fp)

In [3]:
IMAGE_SHAPE = (200, 200, 1)
NUM_CLASSES = len(classes_weight)
NUM_PREDICT = 5
LSTM_NUM_HIDDEN_STATE = 128

# Model acrchitect

1 Image -> ResNet-8 -> LSTM_1 -> LSTM_2 (return state = True) (use this to sample prediction) -> Dense + Softmax 
-> 10 steering angle ID

## 1.2 Create Encoder
Encoder is body of ResNet-8 from `Drone-Net` since it's already trained to recognize road curve -> helpful spatial information


In [4]:
encoder = resnet8_body(IMAGE_SHAPE)
# load resnet8 weights
encoder.load_weights("./s1p10_model/named_resnet8_best_weights.h5", by_name=True)
for l in encoder.layers:
    l.trainable = False
encoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
bn_1_a (BatchNorma

## 1.3 Create Decoder
Decoder is comprised of 2 LSTM & 2 Dense layers, last Dense layer is activated by Softmax function

In [5]:
LSTM_cell = LSTM(LSTM_NUM_HIDDEN_STATE, return_state=True)  # use to sample angle

In [6]:
def dense_classifier(input_shape, num_classes):
    """
    Define classifier made of several Dense layers
    """
    X_in = Input(shape=input_shape)
    
    X = Dense(128)(X_in)
    
    X = Dropout(0.5)(X)
    
    X = PReLU()(X)  # use PReLU activation
    
    X = Dense(64)(X)
    
    X = Dropout(0.5)(X)
    
    X = PReLU()(X)
    
    y = Dense(num_classes, activation='softmax')(X)
    
    model = Model(inputs=[X_in], outputs=[y], name='classifier')
    return model
    

In [7]:
classifier = dense_classifier((LSTM_NUM_HIDDEN_STATE, ), NUM_CLASSES)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 128)               128       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 64)                64        
__________

## 1.4 Create Model

In [9]:
def s1p10_model(input_shape, encoder, LSTM_cell, classifier, lstm_num_hidden_state, num_classes, num_labels):
    """
    Define see-1-predict-10 model
    
    Input:
        input_shape (tuple): shape of input image inputted to encoder
        encoder (keras.Model)
        LSTM_cell (keras.layers)
        classifier (keras.Model)
        
    Output:
        keras.Model
    """
    X_in = Input(shape=input_shape, name="image_in")
    
    # extract feature vector
    X_feature = encoder(X_in)
    X_feature = Reshape((1, -1))(X_feature)  # shape = (None, 1, 6272)
    
    # Initialize input to LSTM
    a0 = Input(shape=(LSTM_NUM_HIDDEN_STATE, ), name="a0")
    c0 = Input(shape=(LSTM_NUM_HIDDEN_STATE, ), name="c0")
    y0 = Input(shape=(num_classes, ), name="y0")  # will be concatenated with X_feature to make X
    
    a = a0
    c = c0
    y = y0
    
    outputs = []
    
    for i in range(num_labels):
        # concatenate y & X_feature
        y = Reshape((1, -1))(y)
        X = keras.layers.concatenate([X_feature, y], axis=-1)
        # propagate X through LSTM_cell
        a, _, c = LSTM_cell(X, initial_state=[a, c])
        # propagate hidden state "a" through classifier to get steering angle
        y = classifier(a)
        # store y
        outputs.append(y)
    
    model = Model(inputs=[X_in, a0, c0, y0], outputs=outputs)
    return model
        

In [10]:
model = s1p10_model(IMAGE_SHAPE, 
                    encoder, 
                    LSTM_cell, 
                    classifier, 
                    LSTM_NUM_HIDDEN_STATE, 
                    NUM_CLASSES, 
                    NUM_PREDICT)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_in (InputLayer)           (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
resnet8 (Model)                 (None, 6272)         309088      image_in[0][0]                   
__________________________________________________________________________________________________
y0 (InputLayer)                 (None, 114)          0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 6272)      0           resnet8[1][0]                    
__________________________________________________________________________________________________
reshape_2 

In [11]:
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

# Training

In [12]:
batch_size = 64

gen_param = {'data_root_dir': "/home/user/Bureau/Dataset/udacity/", 
             'img_shape': IMAGE_SHAPE, 
             'num_class': NUM_CLASSES, 
             'num_prediction': NUM_PREDICT, 
             'batch_size': batch_size, 
             'shuffle': True, 
             'lstm_dim_hidden_states': LSTM_NUM_HIDDEN_STATE}

train_gen = DataGenerator("./s1p10_data/s1p10_CH2_002_output_training.csv", **gen_param)
val_gen = DataGenerator("./s1p10_data/s1p10_CH2_002_output_validation.csv", **gen_param)

In [13]:
time_str = time.strftime("%Y_%m_%d_%H_%M")
log_dir = './s1p10_logs/' + time_str
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')

In [14]:
model.fit_generator(train_gen,
                    class_weight=classes_weight,
                    epochs=20,
                    validation_data=val_gen,
                    initial_epoch=0,
                    callbacks=[tb_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
284/284 [==============================] - 111s 392ms/step - loss: 15.3273 - classifier_loss: 3.0582 - classifier_acc: 0.2197 - classifier_acc_1: 0.2172 - classifier_acc_2: 0.2179 - classifier_acc_3: 0.2159 - classifier_acc_4: 0.2199 - val_loss: 16.7937 - val_classifier_loss: 3.3248 - val_classifier_acc: 0.2116 - val_classifier_acc_1: 0.2117 - val_classifier_acc_2: 0.2082 - val_classifier_acc_3: 0.2074 - val_classifier_acc_4: 0.2086
Epoch 2/20
284/284 [==============================] - 103s 364ms/step - loss: 13.4305 - classifier_loss: 2.6762 - classifier_acc: 0.2524 - classifier_acc_1: 0.2512 - classifier_acc_2: 0.2538 - classifier_acc_3: 0.2503 - classifier_acc_4: 0.2545 - val_loss: 17.2256 - val_classifier_loss: 3.4434 - val_classifier_acc: 0.1765 - val_classifier_acc_1: 0.1715 - val_classifier_acc_2: 0.1682 - val_classifier_acc_3: 0.1692 - val_classifier_acc_4: 0.1691
Epoch 3/20
284/284 [==============================] - 10

Epoch 19/20
284/284 [==============================] - 105s 370ms/step - loss: 9.1746 - classifier_loss: 1.8519 - classifier_acc: 0.3806 - classifier_acc_1: 0.4025 - classifier_acc_2: 0.4130 - classifier_acc_3: 0.4067 - classifier_acc_4: 0.3923 - val_loss: 31.5336 - val_classifier_loss: 6.5638 - val_classifier_acc: 0.0519 - val_classifier_acc_1: 0.0538 - val_classifier_acc_2: 0.0559 - val_classifier_acc_3: 0.0574 - val_classifier_acc_4: 0.0583
Epoch 20/20
284/284 [==============================] - 104s 367ms/step - loss: 9.0056 - classifier_loss: 1.8258 - classifier_acc: 0.3872 - classifier_acc_1: 0.4121 - classifier_acc_2: 0.4245 - classifier_acc_3: 0.4202 - classifier_acc_4: 0.4020 - val_loss: 33.3867 - val_classifier_loss: 6.9565 - val_classifier_acc: 0.0547 - val_classifier_acc_1: 0.0532 - val_classifier_acc_2: 0.0546 - val_classifier_acc_3: 0.0546 - val_classifier_acc_4: 0.0559


# Save weights

In [15]:
# save shared_lstm
save_lstm(LSTM_cell, time_str, log_dir)

# save classifier
classifier.save_weights(log_dir + "/classifier_%s.h5" % time_str)

In [16]:
time_str

'2019_05_27_08_54'